Lo primero que haré es abrir convertir los doas archivos csv en data frame y unirlos en uno solo por que si bien luego lo enviaré a mysql, que en teoria es relacional la información es totalmente complementaria

In [1]:
import pandas as pd

In [2]:
# Este código se usa para poder visualizar todas las filas de un data frame muy largo
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 30)

In [ ]:
# Leo y visualizo df_hechos con la información original

df_hechos = pd.read_csv('hechos.csv', sep=';')
df_hechos

In [ ]:
df_hechos.info()
# Se puede verificar que este df tiene 20 filas

In [ ]:
# Leo y visualizo df_victimas  con la información original

df_victimas = pd.read_csv('victimas.csv', sep=';')
df_victimas 

In [ ]:
df_victimas.info()
# Se puede verificar que son 9 filas, es decir que sumadas serán 29 filas, se utilizará este numero para la configuración de tamaño

In [ ]:
# Cambio el nombre de la columna ID_hechos para poder hacel el join con la informacion víctimas

df_victimas.rename(columns={'ID_hecho': 'ID'}, inplace=True)
df_victimas


In [ ]:
# Unifico y visualizo los dos df anteriores para hacer las transformaciones necesarias y poder mejorar la información
 
df_siniestros = pd.merge(df_hechos, df_victimas, on='ID', how='inner')
df_siniestros

In [ ]:
df_siniestros.info()
# Se pueden ver 29 filas, lo que asegura que en este dataframe quedó la totalidad de la información

Si bien la idea es dejar toda la información en una base de datos, se puede mejorar la presentación pues hay mucha información repetida que se puede dejar en una sola columna con el mismo formato tal como el año, así que iré columna por columna para entender y modificar su forma.

In [ ]:
#Identifico el tipo de columnas (ID) para poder comparar registro y encontrar posteriormente duplicados

tipos_columna_ID = df_siniestros.dtypes
print(tipos_columna_ID)

In [12]:
# Dado que el tipo de la columna ID es objeto primero convertiré en strn para poder extraer los ultimos 4 digitor referentes al indicador

df_siniestros['ID'] = df_siniestros['ID'].astype(str)

In [ ]:
# Ahora si extraeré los ultimos 4 dígitos en una nueva columna "IDENTIFICADOR"

df_siniestros['IDENTIFICADOR'] = df_siniestros['ID'].astype(str).str[-4:]
df_siniestros


In [ ]:
# Dejo la ultima columna de primeras pues por ser el identificador es el mas importante para la organización de los datos

# Cambiar la ubicación de la columna 'C' a la primera posición

df_siniestros = df_siniestros[["IDENTIFICADOR"] + [col for col in df_siniestros.columns if col != "IDENTIFICADOR"]]
df_siniestros

In [ ]:
# Tamaño de dataframe
df_siniestros.shape

Los siguietes analisis columna por columna para el EDA serán:


* Fecha siniestro: esta la columna FECHA_xx tiene formato correcto y completo, y la información es necesaria
* La Columna AAAA_xx que se refiere al año se podrá  extraer en PowerBI después, así que prefiero borrarle, así mismo pasará con  MM_x (8mes) y dd_x (día), por lo que las borraré
* La columna Hora tiene  con un formato correcto y es información relevante, posterior cambiaré todos los títulos
* La columna HH se refiere a la franja horaria, lo cual no tiene mucho sentido con tantos números, será inteligente
dejar la columna como franja horaria  pero dejar solo 3 franjas horarias: mañana, tade y noche o depronto solo AM y PM
* LUGAR_DEL_HECHO: dirección del hecho, strn, lo dejaré pero lo más posible es que no se use pues será mas apropiado las coordenadas para poder ubicar en PBI. La idea de tener la dirección del hecho es poder ubicarlo en el mapa, sinembargo la forma mas facil de ubicacion en PBI son las 
coordenadas, se podrá revisar por barrios pero la dirección no sirve de mucho, así que se podrá borrar
* TIPO_DE_CALLE: Tenendo las coordenadas, esta información es irrelevante por lo anterior la borraré
* ACUSADO. Es una información importante pues rendirá cuenta de que tipo de acusado suele tener mas culpabilidad, 
* VICTIMA: En los dos dataframe hay columnas víctimas y en las dos se refieren al vehículo que opcupaba la víctima; Así que borro una de las columnas víctima y dejo una única columna llamada VICTIMA
* FECHA_x,AAAA_x,MM_x,DD_x	Decido borrar esta columna después de verificar que es exactamente la misma información que la columna FECHA_x desagregada, esa desagregación se puede hacer directamente PBI
* La columna XY(CABA), se refiere a la geocodificacion plana, Se tienen las coordenadas por lo que esta información es irrelevante por lo dejaré y si es necesaria la utilizaré en la visualización
POS
* pos x	pos y: Esta es una información muy importante y se refiere a la longitud y latitud en el formato correcto en el que PBI lo ubica, así que cambaré el nombre de las columnas en ese sentido 


In [ ]:
# Borro columnas analizadas en el aparte anterior: FECHA_x,AAAA_x,MM_x,DD_x

columnas_a_eliminar = ['FECHA_y','AAAA_y', 'MM_y','DD_y', 'ROL', 'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE', 'Altura', 'AAAA_x', 'MM_x','DD_x', 'VICTIMA_y']
df_siniestros = df_siniestros.drop(columns=columnas_a_eliminar, axis=1)
df_siniestros

In [18]:
# Cambio el nombre de las columnas posx y pos por LONGITUD y LATITUD, Dado que verifique algunas direcciones normalizadas
# y encontré la coincidencia con estas columnas

df_siniestros.rename(columns={'pos x'	: 'LATITUD'}, inplace=True)
df_siniestros.rename(columns={'pos y'	: 'LONGITUD'}, inplace=True)

In [ ]:
# Renombro la columna N_VICTIMAS por No_VICTIMAS

df_siniestros.rename(columns={'N_VICTIMAS': 'No_VICTIMAS'}, inplace=True)
df_siniestros

In [ ]:
# Cambio el nombre de la columna FECHA_x por FECHA
df_siniestros.rename(columns={'FECHA_X': 'FECHA_HECHO'}, inplace=True)
df_siniestros

In [ ]:
# Cambio el nombre de la columna FECHA_x por FECHA
df_siniestros.rename(columns={'FECHA_X': 'FECHA_HECHO'}, inplace=True)
df_siniestros

In [ ]:
# Cambio el nombre de la columna Calle por CALLE con el fin de estandarizar los títulos
df_siniestros.rename(columns={'Calle': 'CALLE'}, inplace=True)

# Cambio el nombre de la columna Cruce por CRUCE con el fin de estandarizar los títulos
df_siniestros.rename(columns={'Cruce': 'CRUCE'}, inplace=True)

# Cambio el nombre de la columna Direcci�n Normalizada por CRUCE con el fin de estandarizar los títulos
df_siniestros.rename(columns={'Direcci�n Normalizada': 'DIRECCIÓN'}, inplace=True)

# Cambio el nombre de la columna Direcci�n Normalizada por CRUCE con el fin de estandarizar los títulos
df_siniestros.rename(columns={'VICTIMA_x': 'VICTIMA'}, inplace=True)

df_siniestros


In [24]:
df_siniestros.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30108 entries, 0 to 30107
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   IDENTIFICADOR        30108 non-null  object 
 1   ID                   30108 non-null  object 
 2   No_VICTIMAS          717 non-null    float64
 3   FECHA_x              717 non-null    object 
 4   HORA                 717 non-null    object 
 5   HH                   717 non-null    object 
 6   CALLE                716 non-null    object 
 7   CRUCE                540 non-null    object 
 8   DIRECCIÓN            708 non-null    object 
 9   COMUNA               717 non-null    float64
 10  XY (CABA)            717 non-null    object 
 11  LATITUD              717 non-null    object 
 12  LONGITUD             717 non-null    object 
 13  PARTICIPANTES        717 non-null    object 
 14  VICTIMA              717 non-null    object 
 15  ACUSADO              717 non-null   

In [149]:
#Para continuar con la visualización correcta ordenaré el data frame por año y luego por identificador

df_siniestros = df_siniestros.sort_values(by=['columna1', 'columna2'])

KeyError: 'columna1'

In [10]:
# ID: Identificador contiene año y número, bastaría con el identificador pues hay columna independiente de año, así que primero 
# verifico si existen registros iguales para poder retirar los cuatro digitos del año y dejer solo el numero identificador 
# que deja ventajas como organizar facilmente la información

# Verificar duplicados en la columna 'Columna'
duplicados_df_siniestros = df_siniestros.duplicated(subset='ID')

# Imprimir los registros duplicados
print("Registros duplicados en la columna 'ID':")
print(df_siniestros [duplicados_df_siniestros])


Registros duplicados en la columna 'ID':
              ID  N_VICTIMAS     FECHA_x  AAAA_x  MM_x  DD_x      HORA   HH  \
30     2016-0041         2.0  29/03/2016  2016.0   3.0  29.0  11:00:00   11   
99     2016-0126         2.0  18/09/2016  2016.0   9.0  18.0  22:45:00   22   
164    2017-0026         2.0  26/02/2017  2017.0   2.0  26.0   5:15:00    5   
174    2017-0035         3.0  23/03/2017  2017.0   3.0  23.0   5:00:00    5   
175    2017-0035         3.0  23/03/2017  2017.0   3.0  23.0   5:00:00    5   
...          ...         ...         ...     ...   ...   ...       ...  ...   
30103        NaN         NaN         NaN     NaN   NaN   NaN       NaN  NaN   
30104        NaN         NaN         NaN     NaN   NaN   NaN       NaN  NaN   
30105        NaN         NaN         NaN     NaN   NaN   NaN       NaN  NaN   
30106        NaN         NaN         NaN     NaN   NaN   NaN       NaN  NaN   
30107        NaN         NaN         NaN     NaN   NaN   NaN       NaN  NaN   

          